In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
plt.rc('font', family='NanumBarunGothic') 

cooking = pd.read_csv("/content/drive/MyDrive/Projectdata/Cooking_data_set.csv")
error = pd.read_csv("/content/drive/MyDrive/Projectdata/Error_Message.csv")
product = pd.read_csv("/content/drive/MyDrive/Projectdata/Product_Name.csv")
booking = pd.read_csv("/content/drive/MyDrive/Projectdata/booking_data.csv")

Mounted at /content/drive
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 5 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (6,818 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling 

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


*   수주물량예측
*   **출하여부에 영향을 주는 것을 찾기**



In [7]:
booking

수주사업장      수주일자  수주일련번호   거래처코드   수주부서    수주품목코드 수주단위  출고수량KG  \
0         60.0  20151214     436  SI0275  sc600  1BA03008   EA     1.5   
1         60.0  20151214     437  SI0275  sc600  1BA03059   EA     0.9   
2         60.0  20151214     438  SI0275  sc600  1BA03058   EA     0.9   
3         60.0  20151214     441  SI0298  sc600  16A01039   EA     5.0   
4         60.0  20151214     443  SI0298  sc600  16D01021   kg   100.0   
...        ...       ...     ...     ...    ...       ...  ...     ...   
1757233   60.0  20211102     152  SI0275  sc600  13R10064   EA     0.0   
1757234    1.0  20211104       1  IA1406    400  13A00748   kg     0.0   
1757235    1.0  20211104       2  IA1406    400  18A03045   kg     0.0   
1757236   20.0  20211030     113  IJ9780    400  16A03015   EA     0.0   
1757237   20.0  20211102     386  IA1444    NaN  13D01231   EA     0.0   

         수주수량KG      수주단가       수주금액   부가세금액 출하완료여부      납기일자     기준단가 생산지시  \
0           1.5      0.00        0.0       0      Y  20151215      0.0    N   
1           0.9      0.00        0.0       0      Y  20151215      0.0    N   
2           0.9      0.00        0.0       0      Y  20151215      0.0    N   
3           5.0   3630.00    16500.0    1650      Y  20151215   3630.0    N   
4         100.0   5400.00   540000.0       0      Y  20151215   5400.0    N   
...         ...       ...        ...     ...    ...       ...      ...  ...   
1757233    12.0      0.00        0.0       0      N  20211103      0.0    N   
1757234   200.0   9800.00  1960000.0  196000      N  20211105   9800.0  NaN   
1757235    30.0   8400.00   252000.0   25200      N  20211105   8400.0  NaN   
1757236     1.0   4405.76     4406.0       0      Y  20211031   4410.0    N   
1757237   960.0  12500.00  6000000.0  600000      N  20211103  12500.0    N   

          수주수량  
0          5.0  
1          3.0  
2          3.0  
3          5.0  
4          0.0  
...        ...  
1757233    6.0  
1757234    0.0  
1757235    0.0  
1757236    1.0  
1757237  480.0  

[1757238 rows x 17 columns]

In [4]:
cooking.head(3)

순번 생산라인코드  생산라인명        생산일자      품목코드             품목명 작업장코드      충전실온도  \
0   3   C001  쿠커 #1  2011-01-03  23K01312      서비스면양념장(재)  W003  71.669853   
1   1   C002  쿠커 #2  2011-01-03  23V00019  고구마샐러드믹스베이스(재)  W003  70.386480   
2  13   C003  쿠커 #3  2011-01-03  1.#JE+00   이마트가쓰오다시국물(재)  W003  71.206130   

         실링온도        쿠킹온도  쿠킹스팀압력     실링압력   생산시간  
0   96.769853  131.742003  23.315  201.844  70.37  
1  102.086480  136.926080  23.226  212.043  70.83  
2  112.306130  147.579680  23.516  208.781  73.55

In [5]:
error.head()

순번 생산라인코드    생산라인명        생산일자   발생시간   종료시간  오류조치시간    비가동코드  \
0  10   D004     PET병  2011-01-03  14:16  14:24       8  W0056B4   
1   7   D011  2동 오리히로  2011-01-04  18:54  19:03       9  W005711   
2   2   C002    쿠커 #2  2011-01-05   9:51   9:58       7  W003452   
3  13   P004    PET#1  2011-01-05  18:20  20:44     144  W0076B2   
4  17   C004    쿠커 #4  2011-01-10  22:32   1:16     164  W003721   

  Error_Message      품목코드          품목명 오류발생작업장명  
0          위치조정  13D02080     참깨흑임자드레싱      충전실  
1            대기  16A01016  시아스고구마샐러드믹스      충전실  
2            충전  23K01321   나가사키베이스(재)      쿠킹실  
3           구동부  13R02051  이마트베스트돈까스소스      포장실  
4        충진 미종료  23D02080  참깨흑임자드레싱(재)      쿠킹실

In [8]:
product.head()

품목코드                  품목명  평균수주단가  Column 4
0       NaN                  NaN     NaN       NaN
1  1.#JE+00         가쓰오다시국물900ml     NaN       NaN
2  11501122         조미액-갈릭라이스3kg     NaN       NaN
3  11501131  조미유-쉐프솔루션베이스볶음밥(시즌)     NaN       NaN
4  11501133   조미액-소불고기볶음밥-JB/이랜드     NaN       NaN

In [15]:
booking

수주사업장      수주일자  수주일련번호   거래처코드   수주부서    수주품목코드 수주단위  출고수량KG  \
0         60.0  20151214     436  SI0275  sc600  1BA03008   EA     1.5   
1         60.0  20151214     437  SI0275  sc600  1BA03059   EA     0.9   
2         60.0  20151214     438  SI0275  sc600  1BA03058   EA     0.9   
3         60.0  20151214     441  SI0298  sc600  16A01039   EA     5.0   
4         60.0  20151214     443  SI0298  sc600  16D01021   kg   100.0   
...        ...       ...     ...     ...    ...       ...  ...     ...   
1757233   60.0  20211102     152  SI0275  sc600  13R10064   EA     0.0   
1757234    1.0  20211104       1  IA1406    400  13A00748   kg     0.0   
1757235    1.0  20211104       2  IA1406    400  18A03045   kg     0.0   
1757236   20.0  20211030     113  IJ9780    400  16A03015   EA     0.0   
1757237   20.0  20211102     386  IA1444    NaN  13D01231   EA     0.0   

         수주수량KG      수주단가       수주금액   부가세금액 출하완료여부      납기일자     기준단가 생산지시  \
0           1.5      0.00        0.0       0      Y  20151215      0.0    N   
1           0.9      0.00        0.0       0      Y  20151215      0.0    N   
2           0.9      0.00        0.0       0      Y  20151215      0.0    N   
3           5.0   3630.00    16500.0    1650      Y  20151215   3630.0    N   
4         100.0   5400.00   540000.0       0      Y  20151215   5400.0    N   
...         ...       ...        ...     ...    ...       ...      ...  ...   
1757233    12.0      0.00        0.0       0      N  20211103      0.0    N   
1757234   200.0   9800.00  1960000.0  196000      N  20211105   9800.0  NaN   
1757235    30.0   8400.00   252000.0   25200      N  20211105   8400.0  NaN   
1757236     1.0   4405.76     4406.0       0      Y  20211031   4410.0    N   
1757237   960.0  12500.00  6000000.0  600000      N  20211103  12500.0    N   

          수주수량  
0          5.0  
1          3.0  
2          3.0  
3          5.0  
4          0.0  
...        ...  
1757233    6.0  
1757234    0.0  
1757235    0.0  
1757236    1.0  
1757237  480.0  

[1757238 rows x 17 columns]

In [9]:
booking.isnull().sum()

수주사업장         3
수주일자          0
수주일련번호        0
거래처코드        18
수주부서      76062
수주품목코드       29
수주단위      14810
출고수량KG        0
수주수량KG        0
수주단가          0
수주금액          0
부가세금액         0
출하완료여부        0
납기일자          1
기준단가          0
생산지시      70103
수주수량          0
dtype: int64

In [22]:
booking_set = booking.drop(['수주사업장','수주부서','수주단위','생산지시'], axis=1)
booking_set

수주일자  수주일련번호   거래처코드    수주품목코드  출고수량KG  수주수량KG      수주단가  \
0        20151214     436  SI0275  1BA03008     1.5     1.5      0.00   
1        20151214     437  SI0275  1BA03059     0.9     0.9      0.00   
2        20151214     438  SI0275  1BA03058     0.9     0.9      0.00   
3        20151214     441  SI0298  16A01039     5.0     5.0   3630.00   
4        20151214     443  SI0298  16D01021   100.0   100.0   5400.00   
...           ...     ...     ...       ...     ...     ...       ...   
1757233  20211102     152  SI0275  13R10064     0.0    12.0      0.00   
1757234  20211104       1  IA1406  13A00748     0.0   200.0   9800.00   
1757235  20211104       2  IA1406  18A03045     0.0    30.0   8400.00   
1757236  20211030     113  IJ9780  16A03015     0.0     1.0   4405.76   
1757237  20211102     386  IA1444  13D01231     0.0   960.0  12500.00   

              수주금액   부가세금액 출하완료여부      납기일자     기준단가   수주수량  
0              0.0       0      Y  20151215      0.0    5.0  
1              0.0       0      Y  20151215      0.0    3.0  
2              0.0       0      Y  20151215      0.0    3.0  
3          16500.0    1650      Y  20151215   3630.0    5.0  
4         540000.0       0      Y  20151215   5400.0    0.0  
...            ...     ...    ...       ...      ...    ...  
1757233        0.0       0      N  20211103      0.0    6.0  
1757234  1960000.0  196000      N  20211105   9800.0    0.0  
1757235   252000.0   25200      N  20211105   8400.0    0.0  
1757236     4406.0       0      Y  20211031   4410.0    1.0  
1757237  6000000.0  600000      N  20211103  12500.0  480.0  

[1757238 rows x 13 columns]

In [23]:
booking_set.isnull().sum()

수주일자       0
수주일련번호     0
거래처코드     18
수주품목코드    29
출고수량KG     0
수주수량KG     0
수주단가       0
수주금액       0
부가세금액      0
출하완료여부     0
납기일자       1
기준단가       0
수주수량       0
dtype: int64

In [24]:
booking_set[booking_set['출하완료여부'] == 'Y']['수주수량KG'].sort_values(ascending=False)

702964     20170906.0
1462935     3900000.0
853671      3022500.0
1537490      309412.0
1256724      250387.2
              ...    
551034      -571588.0
551049      -665844.0
551041      -701598.0
551059      -701874.0
551043      -791654.0
Name: 수주수량KG, Length: 1750651, dtype: float64

In [25]:
booking_set[booking_set['출하완료여부'] == 'N']['수주수량KG'].sort_values(ascending=False)

1756376    135355.00
1718888     60000.00
1501399     50000.00
1664720     37240.00
1753331     20999.16
             ...    
1485906         0.00
1729542         0.00
1496732       -48.00
878690       -102.24
878789       -102.24
Name: 수주수량KG, Length: 6587, dtype: float64

In [28]:
booking_set.groupby('출하완료여부')['수주수량KG'].count()

출하완료여부
N       6587
Y    1750651
Name: 수주수량KG, dtype: int64

In [29]:
booking_set

수주일자  수주일련번호   거래처코드    수주품목코드  출고수량KG  수주수량KG      수주단가  \
0        20151214     436  SI0275  1BA03008     1.5     1.5      0.00   
1        20151214     437  SI0275  1BA03059     0.9     0.9      0.00   
2        20151214     438  SI0275  1BA03058     0.9     0.9      0.00   
3        20151214     441  SI0298  16A01039     5.0     5.0   3630.00   
4        20151214     443  SI0298  16D01021   100.0   100.0   5400.00   
...           ...     ...     ...       ...     ...     ...       ...   
1757233  20211102     152  SI0275  13R10064     0.0    12.0      0.00   
1757234  20211104       1  IA1406  13A00748     0.0   200.0   9800.00   
1757235  20211104       2  IA1406  18A03045     0.0    30.0   8400.00   
1757236  20211030     113  IJ9780  16A03015     0.0     1.0   4405.76   
1757237  20211102     386  IA1444  13D01231     0.0   960.0  12500.00   

              수주금액   부가세금액 출하완료여부      납기일자     기준단가   수주수량  
0              0.0       0      Y  20151215      0.0    5.0  
1              0.0       0      Y  20151215      0.0    3.0  
2              0.0       0      Y  20151215      0.0    3.0  
3          16500.0    1650      Y  20151215   3630.0    5.0  
4         540000.0       0      Y  20151215   5400.0    0.0  
...            ...     ...    ...       ...      ...    ...  
1757233        0.0       0      N  20211103      0.0    6.0  
1757234  1960000.0  196000      N  20211105   9800.0    0.0  
1757235   252000.0   25200      N  20211105   8400.0    0.0  
1757236     4406.0       0      Y  20211031   4410.0    1.0  
1757237  6000000.0  600000      N  20211103  12500.0  480.0  

[1757238 rows x 13 columns]

In [61]:
booking_set.isnull().sum()

수주일자       0
수주일련번호     0
거래처코드     18
수주품목코드    29
출고수량KG     0
수주수량KG     0
수주단가       0
수주금액       0
부가세금액      0
출하완료여부     0
납기일자       1
기준단가       0
수주수량       0
dtype: int64

In [56]:
booking_12 = booking[booking['수주수량KG'] >=500]
booking_12

수주사업장      수주일자  수주일련번호   거래처코드   수주부서    수주품목코드 수주단위  출고수량KG  \
5          1.0  20151214       2  IJ9455    400  1S101004   EA   921.6   
124       20.0  20151214     136  IJ9455    400  16B06003   kg  2585.6   
125       20.0  20151214     137  IJ9455    400  16B06084   kg   768.0   
137       20.0  20151214     226  IJ9451    400  16A02010   kg   519.0   
180       20.0  20151213     177  IJ9193    400  16B06003   EA   576.0   
...        ...       ...     ...     ...    ...       ...  ...     ...   
1757209   60.0  20211102     130  SI3119  sc600  13K02080   EA     0.0   
1757227   20.0  20211102     382  IK2217    NaN  1BA03573   EA     0.0   
1757229   20.0  20211102     384  IK2217    NaN  1BA03618   EA     0.0   
1757230   20.0  20211102     385  IK2217    NaN  1BA03584   EA     0.0   
1757237   20.0  20211102     386  IA1444    NaN  13D01231   EA     0.0   

         수주수량KG     수주단가       수주금액   부가세금액 출하완료여부      납기일자     기준단가 생산지시  \
5         921.6   5200.0   567273.0   56727      Y  20151214   5200.0    N   
124      2585.6   1675.0  4330880.0  135340      Y  20151215   1675.0    N   
125       768.0   5345.0  1282800.0  128280      Y  20151215   5345.0    N   
137       519.0   3104.0  1610976.0  161098      Y  20151215   3104.0    N   
180       576.0   6390.0  1150200.0  115020      Y  20151217   6390.0    N   
...         ...      ...        ...     ...    ...       ...      ...  ...   
1757209   600.0   7810.0  2130000.0  213000      N  20211104   7810.0    N   
1757227  2100.0   1100.0  9240000.0  924000      N  20211105   1100.0    N   
1757229  1050.0   1450.0  6090000.0  609000      N  20211105   1450.0    N   
1757230  1400.0   1430.0  8008000.0  800800      N  20211105   1430.0    N   
1757237   960.0  12500.0  6000000.0  600000      N  20211103  12500.0    N   

           수주수량  
5         120.0  
124         0.0  
125         0.0  
137         0.0  
180       180.0  
...         ...  
1757209   300.0  
1757227  8400.0  
1757229  4200.0  
1757230  5600.0  
1757237   480.0  

[145155 rows x 17 columns]

In [47]:
booking_12[booking_12['출하완료여부'] == 'Y']['수주수량KG'].sort_values(ascending=False)

702964     20170906.0
1462935     3900000.0
853671      3022500.0
1537490      309412.0
1256724      250387.2
              ...    
157426         1000.0
649636         1000.0
909878         1000.0
1162342        1000.0
1073305        1000.0
Name: 수주수량KG, Length: 73035, dtype: float64

In [48]:
booking_12[booking_12['출하완료여부'] == 'N']['수주수량KG'].sort_values(ascending=False)

1756376    135355.00
1718888     60000.00
1501399     50000.00
1664720     37240.00
1753331     20999.16
             ...    
1650358      1000.00
1545922      1000.00
1547169      1000.00
1753412      1000.00
1717897      1000.00
Name: 수주수량KG, Length: 613, dtype: float64

In [55]:
booking_set.groupby('출하완료여부')['출고수량KG','수주수량KG','수주수량'].mean()

<ipython-input-55-e22bc5438e75>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  booking_set.groupby('출하완료여부')['출고수량KG','수주수량KG','수주수량'].mean()


출고수량KG      수주수량KG        수주수량
출하완료여부                                    
N       205.544096  416.329468  841.041142
Y       181.073454  212.524137  143.424394

In [62]:
booking_set.pivot_table(values='출하완료여부', index=['출고수량KG','수주수량KG','수주수량'])

/usr/local/lib/python3.8/dist-packages/pandas/core/apply.py:507: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  return self._try_aggregate_string_function(obj, f, *self.args, **self.kwargs)


Empty DataFrame
Columns: []
Index: [(-55344.0, 55344.0, 0.0), (-600.0, 4800.0, 0.0), (-344.0, 344.0, 800.0), (-28.8, 28.8, 72.0), (-27.36, 27.36, 72.0), (-22.8, 22.8, 60.0), (-15.12, 15.12, 9.0), (-15.0, 18.0, 0.0), (-10.8, 10.8, 12.0), (-9.12, 9.12, 24.0), (-1.0, 1.0, 0.0), (0.0, -791654.0, 0.0), (0.0, -701874.0, 0.0), (0.0, -701598.0, 0.0), (0.0, -665844.0, 0.0), (0.0, -571588.0, 0.0), (0.0, -435442.0, 0.0), (0.0, -421460.0, 0.0), (0.0, -369442.0, 0.0), (0.0, -362660.0, 0.0), (0.0, -357164.0, 0.0), (0.0, -347864.0, 0.0), (0.0, -227894.0, 0.0), (0.0, -138466.0, 0.0), (0.0, -87260.0, 0.0), (0.0, -44778.0, 0.0), (0.0, -43464.0, 0.0), (0.0, -42192.0, 0.0), (0.0, -32770.0, 0.0), (0.0, -31582.0, 0.0), (0.0, -7444.0, 0.0), (0.0, -6626.0, 0.0), (0.0, -6324.0, 0.0), (0.0, -4904.0, 0.0), (0.0, -3840.0, 0.0), (0.0, -1407.0, 0.0), (0.0, -966.0, 0.0), (0.0, -819.0, 0.0), (0.0, -761.1, 0.0), (0.0, -745.8, 0.0), (0.0, -741.6, 0.0), (0.0, -720.0, 0.0), (0.0, -658.05, 0.0), (0.0, -600.0, 0.0), (0.0, -477.0, 0.0), (0.0, -453.0, 0.0), (0.0, -380.0, 0.0), (0.0, -250.0, 0.0), (0.0, -232.5, 0.0), (0.0, -230.4, 0.0), (0.0, -194.49, 0.0), (0.0, -190.6112, 0.0), (0.0, -163.5616, 0.0), (0.0, -161.0, 0.0), (0.0, -160.0, 0.0), (0.0, -147.0, 0.0), (0.0, -146.88, 0.0), (0.0, -136.8, 0.0), (0.0, -110.0, 0.0), (0.0, -102.24, 0.0), (0.0, -76.5984, 0.0), (0.0, -71.424, 0.0), (0.0, -70.0256, 0.0), (0.0, -63.36, 0.0), (0.0, -57.6, 0.0), (0.0, -56.28, 0.0), (0.0, -50.0, -50.0), (0.0, -50.0, 0.0), (0.0, -48.0, 0.0), (0.0, -47.616, 0.0), (0.0, -46.305, 0.0), (0.0, -42.0, 0.0), (0.0, -40.0, 0.0), (0.0, -39.0, 0.0), (0.0, -38.88, 0.0), (0.0, -38.1728, 0.0), (0.0, -36.0, 0.0), (0.0, -34.0, 0.0), (0.0, -30.0, 0.0), (0.0, -28.0, 0.0), (0.0, -26.0, 0.0), (0.0, -21.2352, 0.0), (0.0, -18.0, 0.0), (0.0, -16.0, 0.0), (0.0, -14.4, 0.0), (0.0, -14.0, 0.0), (0.0, -12.6, -6.0), (0.0, -12.0, 0.0), (0.0, -11.7, 0.0), (0.0, -10.0, 0.0), (0.0, -8.28, 0.0), (0.0, -8.0, 0.0), (0.0, -7.125, 0.0), (0.0, -6.0, 0.0), (0.0, -5.0, 0.0), (0.0, -4.8032, 0.0), (0.0, -4.0, 0.0), (0.0, -3.185, 0.0), (0.0, -1.715, 0.0), (0.0, -1.0, 0.0), ...]

[98170 rows x 0 columns]

In [63]:
booking

수주사업장      수주일자  수주일련번호   거래처코드   수주부서    수주품목코드 수주단위  출고수량KG  \
0         60.0  20151214     436  SI0275  sc600  1BA03008   EA     1.5   
1         60.0  20151214     437  SI0275  sc600  1BA03059   EA     0.9   
2         60.0  20151214     438  SI0275  sc600  1BA03058   EA     0.9   
3         60.0  20151214     441  SI0298  sc600  16A01039   EA     5.0   
4         60.0  20151214     443  SI0298  sc600  16D01021   kg   100.0   
...        ...       ...     ...     ...    ...       ...  ...     ...   
1757233   60.0  20211102     152  SI0275  sc600  13R10064   EA     0.0   
1757234    1.0  20211104       1  IA1406    400  13A00748   kg     0.0   
1757235    1.0  20211104       2  IA1406    400  18A03045   kg     0.0   
1757236   20.0  20211030     113  IJ9780    400  16A03015   EA     0.0   
1757237   20.0  20211102     386  IA1444    NaN  13D01231   EA     0.0   

         수주수량KG      수주단가       수주금액   부가세금액 출하완료여부      납기일자     기준단가 생산지시  \
0           1.5      0.00        0.0       0      Y  20151215      0.0    N   
1           0.9      0.00        0.0       0      Y  20151215      0.0    N   
2           0.9      0.00        0.0       0      Y  20151215      0.0    N   
3           5.0   3630.00    16500.0    1650      Y  20151215   3630.0    N   
4         100.0   5400.00   540000.0       0      Y  20151215   5400.0    N   
...         ...       ...        ...     ...    ...       ...      ...  ...   
1757233    12.0      0.00        0.0       0      N  20211103      0.0    N   
1757234   200.0   9800.00  1960000.0  196000      N  20211105   9800.0  NaN   
1757235    30.0   8400.00   252000.0   25200      N  20211105   8400.0  NaN   
1757236     1.0   4405.76     4406.0       0      Y  20211031   4410.0    N   
1757237   960.0  12500.00  6000000.0  600000      N  20211103  12500.0    N   

          수주수량  
0          5.0  
1          3.0  
2          3.0  
3          5.0  
4          0.0  
...        ...  
1757233    6.0  
1757234    0.0  
1757235    0.0  
1757236    1.0  
1757237  480.0  

[1757238 rows x 17 columns]

In [67]:
booking.groupby('수주단위')['수주수량'].mean()

수주단위
EA    220.266009
kg      8.131767
Name: 수주수량, dtype: float64

In [71]:
booking.pivot_table(index='수주단위', values='수주수량')

수주수량
수주단위            
EA    220.266009
kg      8.131767

In [76]:
booking_kg = booking[booking['수주단위'] == 'kg']
booking_kg

수주사업장      수주일자  수주일련번호   거래처코드   수주부서    수주품목코드 수주단위  출고수량KG  \
4         60.0  20151214     443  SI0298  sc600  16D01021   kg   100.0   
54        20.0  20151213     173  IJ1747    400  13P01241   kg    23.0   
96        20.0  20151214     238  IJ9451    400  13K01115   kg    12.0   
97        20.0  20151214     239  IJ9451    400  13D08030   kg     2.0   
98        20.0  20151214     240  IJ9451    400  1BA03067   kg   300.0   
...        ...       ...     ...     ...    ...       ...  ...     ...   
1757205   20.0  20211102     379  IK0330    NaN  13A00702   kg     0.0   
1757206   20.0  20211102     380  IK0330    NaN  13A00701   kg     0.0   
1757207   20.0  20211102     381  IK0330    NaN  13D01307   kg     0.0   
1757234    1.0  20211104       1  IA1406    400  13A00748   kg     0.0   
1757235    1.0  20211104       2  IA1406    400  18A03045   kg     0.0   

         수주수량KG      수주단가       수주금액   부가세금액 출하완료여부      납기일자      기준단가 생산지시  \
4         100.0   5400.00   540000.0       0      Y  20151215   5400.00    N   
54         23.0   6930.00   159390.0   15939      Y  20151215   6930.00    N   
96         12.0   3621.50    43458.0    2173      Y  20151215   3621.50    N   
97          2.0   3575.00     7150.0     358      Y  20151215   3575.00    N   
98        300.0  10600.00  1060000.0  106000      Y  20151215  10600.00    N   
...         ...       ...        ...     ...    ...       ...       ...  ...   
1757205   360.0   2850.00  1026000.0  102600      N  20211103   2850.00    N   
1757206   960.0   2753.33  2643196.0  264319      N  20211103   2753.33    N   
1757207    12.0   6410.00    76920.0    7692      N  20211103   6410.00    N   
1757234   200.0   9800.00  1960000.0  196000      N  20211105   9800.00  NaN   
1757235    30.0   8400.00   252000.0   25200      N  20211105   8400.00  NaN   

          수주수량  
4          0.0  
54         0.0  
96         0.0  
97         0.0  
98         0.0  
...        ...  
1757205  360.0  
1757206  960.0  
1757207   12.0  
1757234    0.0  
1757235    0.0  

[599525 rows x 17 columns]

In [79]:
booking.dtypes

수주사업장     float64
수주일자        int64
수주일련번호      int64
거래처코드      object
수주부서       object
수주품목코드     object
수주단위       object
출고수량KG    float64
수주수량KG    float64
수주단가      float64
수주금액      float64
부가세금액       int64
출하완료여부     object
납기일자       object
기준단가      float64
생산지시       object
수주수량      float64
dtype: object

In [83]:
booking['수주수량'] = booking['수주수량'].astype({'수주수량':'int'})
booking

수주사업장      수주일자  수주일련번호   거래처코드   수주부서    수주품목코드 수주단위  출고수량KG  \
0         60.0  20151214     436  SI0275  sc600  1BA03008   EA     1.5   
1         60.0  20151214     437  SI0275  sc600  1BA03059   EA     0.9   
2         60.0  20151214     438  SI0275  sc600  1BA03058   EA     0.9   
3         60.0  20151214     441  SI0298  sc600  16A01039   EA     5.0   
4         60.0  20151214     443  SI0298  sc600  16D01021   kg   100.0   
...        ...       ...     ...     ...    ...       ...  ...     ...   
1757233   60.0  20211102     152  SI0275  sc600  13R10064   EA     0.0   
1757234    1.0  20211104       1  IA1406    400  13A00748   kg     0.0   
1757235    1.0  20211104       2  IA1406    400  18A03045   kg     0.0   
1757236   20.0  20211030     113  IJ9780    400  16A03015   EA     0.0   
1757237   20.0  20211102     386  IA1444    NaN  13D01231   EA     0.0   

         수주수량KG      수주단가       수주금액   부가세금액 출하완료여부      납기일자     기준단가 생산지시  \
0           1.5      0.00        0.0       0      Y  20151215      0.0    N   
1           0.9      0.00        0.0       0      Y  20151215      0.0    N   
2           0.9      0.00        0.0       0      Y  20151215      0.0    N   
3           5.0   3630.00    16500.0    1650      Y  20151215   3630.0    N   
4         100.0   5400.00   540000.0       0      Y  20151215   5400.0    N   
...         ...       ...        ...     ...    ...       ...      ...  ...   
1757233    12.0      0.00        0.0       0      N  20211103      0.0    N   
1757234   200.0   9800.00  1960000.0  196000      N  20211105   9800.0  NaN   
1757235    30.0   8400.00   252000.0   25200      N  20211105   8400.0  NaN   
1757236     1.0   4405.76     4406.0       0      Y  20211031   4410.0    N   
1757237   960.0  12500.00  6000000.0  600000      N  20211103  12500.0    N   

         수주수량  
0           5  
1           3  
2           3  
3           5  
4           0  
...       ...  
1757233     6  
1757234     0  
1757235     0  
1757236     1  
1757237   480  

[1757238 rows x 17 columns]

In [84]:
booking['생산지시'].unique()

array(['N', nan, 'Y'], dtype=object)

In [86]:
booking_set = booking.drop('생산지시', axis=1)
booking_set

수주사업장      수주일자  수주일련번호   거래처코드   수주부서    수주품목코드 수주단위  출고수량KG  \
0         60.0  20151214     436  SI0275  sc600  1BA03008   EA     1.5   
1         60.0  20151214     437  SI0275  sc600  1BA03059   EA     0.9   
2         60.0  20151214     438  SI0275  sc600  1BA03058   EA     0.9   
3         60.0  20151214     441  SI0298  sc600  16A01039   EA     5.0   
4         60.0  20151214     443  SI0298  sc600  16D01021   kg   100.0   
...        ...       ...     ...     ...    ...       ...  ...     ...   
1757233   60.0  20211102     152  SI0275  sc600  13R10064   EA     0.0   
1757234    1.0  20211104       1  IA1406    400  13A00748   kg     0.0   
1757235    1.0  20211104       2  IA1406    400  18A03045   kg     0.0   
1757236   20.0  20211030     113  IJ9780    400  16A03015   EA     0.0   
1757237   20.0  20211102     386  IA1444    NaN  13D01231   EA     0.0   

         수주수량KG      수주단가       수주금액   부가세금액 출하완료여부      납기일자     기준단가  수주수량  
0           1.5      0.00        0.0       0      Y  20151215      0.0     5  
1           0.9      0.00        0.0       0      Y  20151215      0.0     3  
2           0.9      0.00        0.0       0      Y  20151215      0.0     3  
3           5.0   3630.00    16500.0    1650      Y  20151215   3630.0     5  
4         100.0   5400.00   540000.0       0      Y  20151215   5400.0     0  
...         ...       ...        ...     ...    ...       ...      ...   ...  
1757233    12.0      0.00        0.0       0      N  20211103      0.0     6  
1757234   200.0   9800.00  1960000.0  196000      N  20211105   9800.0     0  
1757235    30.0   8400.00   252000.0   25200      N  20211105   8400.0     0  
1757236     1.0   4405.76     4406.0       0      Y  20211031   4410.0     1  
1757237   960.0  12500.00  6000000.0  600000      N  20211103  12500.0   480  

[1757238 rows x 16 columns]

In [82]:
booking.isnull().sum()

수주사업장         3
수주일자          0
수주일련번호        0
거래처코드        18
수주부서      76062
수주품목코드       29
수주단위      14810
출고수량KG        0
수주수량KG        0
수주단가          0
수주금액          0
부가세금액         0
출하완료여부        0
납기일자          1
기준단가          0
생산지시      70103
수주수량          0
dtype: int64

In [87]:
booking_set_kg = booking_set[booking_set['수주단위'] == 'kg']
booking_set_kg

수주사업장      수주일자  수주일련번호   거래처코드   수주부서    수주품목코드 수주단위  출고수량KG  \
4         60.0  20151214     443  SI0298  sc600  16D01021   kg   100.0   
54        20.0  20151213     173  IJ1747    400  13P01241   kg    23.0   
96        20.0  20151214     238  IJ9451    400  13K01115   kg    12.0   
97        20.0  20151214     239  IJ9451    400  13D08030   kg     2.0   
98        20.0  20151214     240  IJ9451    400  1BA03067   kg   300.0   
...        ...       ...     ...     ...    ...       ...  ...     ...   
1757205   20.0  20211102     379  IK0330    NaN  13A00702   kg     0.0   
1757206   20.0  20211102     380  IK0330    NaN  13A00701   kg     0.0   
1757207   20.0  20211102     381  IK0330    NaN  13D01307   kg     0.0   
1757234    1.0  20211104       1  IA1406    400  13A00748   kg     0.0   
1757235    1.0  20211104       2  IA1406    400  18A03045   kg     0.0   

         수주수량KG      수주단가       수주금액   부가세금액 출하완료여부      납기일자      기준단가  수주수량  
4         100.0   5400.00   540000.0       0      Y  20151215   5400.00     0  
54         23.0   6930.00   159390.0   15939      Y  20151215   6930.00     0  
96         12.0   3621.50    43458.0    2173      Y  20151215   3621.50     0  
97          2.0   3575.00     7150.0     358      Y  20151215   3575.00     0  
98        300.0  10600.00  1060000.0  106000      Y  20151215  10600.00     0  
...         ...       ...        ...     ...    ...       ...       ...   ...  
1757205   360.0   2850.00  1026000.0  102600      N  20211103   2850.00   360  
1757206   960.0   2753.33  2643196.0  264319      N  20211103   2753.33   960  
1757207    12.0   6410.00    76920.0    7692      N  20211103   6410.00    12  
1757234   200.0   9800.00  1960000.0  196000      N  20211105   9800.00     0  
1757235    30.0   8400.00   252000.0   25200      N  20211105   8400.00     0  

[599525 rows x 16 columns]

In [90]:
booking.index

RangeIndex(start=0, stop=1757238, step=1)

In [88]:
booking_set_kg['수주수량'].unique()

array([    0,    30,   720,   600,  1500,   900,   840,   200,   500,
          60,     2,     4,     9,    80,     5,   240,   400,   612,
          20,   160,  2100,    50,  1000,   480,  2400,    48,    70,
         700,   800,   360,   120,  1200,  3600,   310,  1800,    40,
         280,    24,  1700,   132,   180,   105,    36,    38,    12,
          10,  2000,   250,  2004,     8,   300,    84,    72,    35,
         324,  1224,   432,  1080,   504,  3000,  1320,   420,   100,
         960,  2160,  1008,  2760,   210,   230,   170,  1440,     6,
          27,  1920,    90,  1560,  1020,  2280,   336,   260,  2500,
         540,   138,   470,   264,   150,   450,   560,  2040,   144,
         558,   108,   110,    75,   278,    22,    66,     3,    83,
          55,    25,    15,    11,    52,    16,    64,     1,  3080,
         312,   984,   940,  3120,   440,    96,  1620,   576,  3360,
          63,    14,    17,    28,    33,    21,    45,    37,    34,
         251,     7,

In [93]:
for i, row in booking_set_kg.iterrows():
  if booking_set_kg.at[i, '수주수량'] ==booking_set_kg.at[i, '수주수량KG']:
    booking_set_kg.at[i, '차이'] = 2

booking_set_kg

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


수주사업장      수주일자  수주일련번호   거래처코드   수주부서    수주품목코드 수주단위  출고수량KG  \
4         60.0  20151214     443  SI0298  sc600  16D01021   kg   100.0   
54        20.0  20151213     173  IJ1747    400  13P01241   kg    23.0   
96        20.0  20151214     238  IJ9451    400  13K01115   kg    12.0   
97        20.0  20151214     239  IJ9451    400  13D08030   kg     2.0   
98        20.0  20151214     240  IJ9451    400  1BA03067   kg   300.0   
...        ...       ...     ...     ...    ...       ...  ...     ...   
1757205   20.0  20211102     379  IK0330    NaN  13A00702   kg     0.0   
1757206   20.0  20211102     380  IK0330    NaN  13A00701   kg     0.0   
1757207   20.0  20211102     381  IK0330    NaN  13D01307   kg     0.0   
1757234    1.0  20211104       1  IA1406    400  13A00748   kg     0.0   
1757235    1.0  20211104       2  IA1406    400  18A03045   kg     0.0   

         수주수량KG      수주단가       수주금액   부가세금액 출하완료여부      납기일자      기준단가  수주수량  \
4         100.0   5400.00   540000.0       0      Y  20151215   5400.00     0   
54         23.0   6930.00   159390.0   15939      Y  20151215   6930.00     0   
96         12.0   3621.50    43458.0    2173      Y  20151215   3621.50     0   
97          2.0   3575.00     7150.0     358      Y  20151215   3575.00     0   
98        300.0  10600.00  1060000.0  106000      Y  20151215  10600.00     0   
...         ...       ...        ...     ...    ...       ...       ...   ...   
1757205   360.0   2850.00  1026000.0  102600      N  20211103   2850.00   360   
1757206   960.0   2753.33  2643196.0  264319      N  20211103   2753.33   960   
1757207    12.0   6410.00    76920.0    7692      N  20211103   6410.00    12   
1757234   200.0   9800.00  1960000.0  196000      N  20211105   9800.00     0   
1757235    30.0   8400.00   252000.0   25200      N  20211105   8400.00     0   

          차이  
4        NaN  
54       NaN  
96       NaN  
97       NaN  
98       NaN  
...      ...  
1757205  2.0  
1757206  2.0  
1757207  2.0  
1757234  NaN  
1757235  NaN  

[599525 rows x 17 columns]

In [95]:
booking_set_kg.isnull().sum()

수주사업장          0
수주일자           0
수주일련번호         0
거래처코드          5
수주부서       12338
수주품목코드         3
수주단위           0
출고수량KG         0
수주수량KG         0
수주단가           0
수주금액           0
부가세금액          0
출하완료여부         0
납기일자           1
기준단가           0
수주수량           0
차이        587405
dtype: int64

In [97]:
booking_set_kg = booking_set_kg.drop(['거래처코드','수주부서','수주품목코드','납기일자'], axis=1)
booking_set_kg

수주사업장      수주일자  수주일련번호 수주단위  출고수량KG  수주수량KG      수주단가       수주금액  \
4         60.0  20151214     443   kg   100.0   100.0   5400.00   540000.0   
54        20.0  20151213     173   kg    23.0    23.0   6930.00   159390.0   
96        20.0  20151214     238   kg    12.0    12.0   3621.50    43458.0   
97        20.0  20151214     239   kg     2.0     2.0   3575.00     7150.0   
98        20.0  20151214     240   kg   300.0   300.0  10600.00  1060000.0   
...        ...       ...     ...  ...     ...     ...       ...        ...   
1757205   20.0  20211102     379   kg     0.0   360.0   2850.00  1026000.0   
1757206   20.0  20211102     380   kg     0.0   960.0   2753.33  2643196.0   
1757207   20.0  20211102     381   kg     0.0    12.0   6410.00    76920.0   
1757234    1.0  20211104       1   kg     0.0   200.0   9800.00  1960000.0   
1757235    1.0  20211104       2   kg     0.0    30.0   8400.00   252000.0   

          부가세금액 출하완료여부      기준단가  수주수량   차이  
4             0      Y   5400.00     0  NaN  
54        15939      Y   6930.00     0  NaN  
96         2173      Y   3621.50     0  NaN  
97          358      Y   3575.00     0  NaN  
98       106000      Y  10600.00     0  NaN  
...         ...    ...       ...   ...  ...  
1757205  102600      N   2850.00   360  2.0  
1757206  264319      N   2753.33   960  2.0  
1757207    7692      N   6410.00    12  2.0  
1757234  196000      N   9800.00     0  NaN  
1757235   25200      N   8400.00     0  NaN  

[599525 rows x 13 columns]

In [98]:
booking_set_kg.dropna()

수주사업장      수주일자  수주일련번호 수주단위  출고수량KG  수주수량KG     수주단가       수주금액  \
334       60.0  20200521     124   kg     0.0    30.0  5200.00   156000.0   
335       20.0  20200521     237   kg   720.0   720.0  3800.00  2736000.0   
336       20.0  20200521     238   kg   600.0   600.0  3500.00  2100000.0   
954       20.0  20200522      89   kg  1500.0  1500.0  1750.00  2625000.0   
955       20.0  20200522      90   kg   900.0   900.0  2375.00  2137500.0   
...        ...       ...     ...  ...     ...     ...      ...        ...   
1757202   20.0  20211102     376   kg     0.0   180.0  3150.00   567000.0   
1757204   20.0  20211102     378   kg     0.0   300.0  2750.00   825000.0   
1757205   20.0  20211102     379   kg     0.0   360.0  2850.00  1026000.0   
1757206   20.0  20211102     380   kg     0.0   960.0  2753.33  2643196.0   
1757207   20.0  20211102     381   kg     0.0    12.0  6410.00    76920.0   

          부가세금액 출하완료여부     기준단가  수주수량   차이  
334       15600      Y  5200.00    30  2.0  
335      273600      Y  3800.00   720  2.0  
336      210000      Y  3500.00   600  2.0  
954      262500      Y  1750.00  1500  2.0  
955      213750      Y  2375.00   900  2.0  
...         ...    ...      ...   ...  ...  
1757202   56700      N  3150.00   180  2.0  
1757204   82500      N  2750.00   300  2.0  
1757205  102600      N  2850.00   360  2.0  
1757206  264319      N  2753.33   960  2.0  
1757207    7692      N  6410.00    12  2.0  

[12120 rows x 13 columns]

In [100]:
booking_set_kg['수주수량'] = 0

In [101]:
booking_set_kg

수주사업장      수주일자  수주일련번호 수주단위  출고수량KG  수주수량KG      수주단가       수주금액  \
4         60.0  20151214     443   kg   100.0   100.0   5400.00   540000.0   
54        20.0  20151213     173   kg    23.0    23.0   6930.00   159390.0   
96        20.0  20151214     238   kg    12.0    12.0   3621.50    43458.0   
97        20.0  20151214     239   kg     2.0     2.0   3575.00     7150.0   
98        20.0  20151214     240   kg   300.0   300.0  10600.00  1060000.0   
...        ...       ...     ...  ...     ...     ...       ...        ...   
1757205   20.0  20211102     379   kg     0.0   360.0   2850.00  1026000.0   
1757206   20.0  20211102     380   kg     0.0   960.0   2753.33  2643196.0   
1757207   20.0  20211102     381   kg     0.0    12.0   6410.00    76920.0   
1757234    1.0  20211104       1   kg     0.0   200.0   9800.00  1960000.0   
1757235    1.0  20211104       2   kg     0.0    30.0   8400.00   252000.0   

          부가세금액 출하완료여부      기준단가  수주수량   차이  
4             0      Y   5400.00     0  NaN  
54        15939      Y   6930.00     0  NaN  
96         2173      Y   3621.50     0  NaN  
97          358      Y   3575.00     0  NaN  
98       106000      Y  10600.00     0  NaN  
...         ...    ...       ...   ...  ...  
1757205  102600      N   2850.00     0  2.0  
1757206  264319      N   2753.33     0  2.0  
1757207    7692      N   6410.00     0  2.0  
1757234  196000      N   9800.00     0  NaN  
1757235   25200      N   8400.00     0  NaN  

[599525 rows x 13 columns]